In [1]:
from matplotlib import pyplot as plt
import sys
import requests
import base64
import json
from pprint import pprint
from dotenv import load_dotenv
import os
import time

sys.path.append('../..')
load_dotenv()
username = os.getenv('SQLPAD_USERNAME')
password = os.getenv('PASSWORD')
print(os.getcwd())
print(username, password)

C:\Users\jepid\PycharmProjects\HydraDX-simulations\hydradx\notebooks\Misc


In [14]:
chunk_size = 10000
chunks_per_file = 100

# Encode the username and password in Base64
credentials = f"{username}:{password}"
encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')

headers = {
    'Authorization': f'Basic {encoded_credentials}',
    'Content-Type': 'application/json'  # This is typically required for JSON payloads
}


def check_errors(all_data):
    errors = []
    for i in range(int(len(all_data) / chunk_size)):
        correctIndex = (i + len(errors)) * chunk_size
        if correctIndex >= len(all_data):
            break
        if all_data[i * chunk_size][0] != correctIndex:
            errors.append(i)
    return errors


def add_data(position: int, all_data: list):
    request = {
        'connectionId': "4a34594e-efa6-4f6e-a594-655ca20f2881",
        'batchText': (
            f"with hdx_changes as ("
            f"  select"
            f"    block_id,"
            f"    '0' as asset_id,"
            f"    (args->>'amount')::numeric as amount"
            f"  from event"
            f"  where"
            f"    name like 'Balances.Transfer'"
            f"    and args->>'to' = '0x6d6f646c6f6d6e69706f6f6c0000000000000000000000000000000000000000'"
            f"  union all"
            f"  select"
            f"    block_id,"
            f"    '0' as asset_id,"
            f"    -(args->>'amount')::numeric as amount"
            f"  from event"
            f"  where"
            f"    name like 'Balances.Transfer'"
            f"    and args->>'from' = '0x6d6f646c6f6d6e69706f6f6c0000000000000000000000000000000000000000'"
            f"),"
            f"tokens_changes as ("
            f"  select"
            f"    block_id,"
            f"    args->>'currencyId' as asset_id,"
            f"    (args->>'amount')::numeric as amount"
            f"  from event"
            f"  where"
            f"    name = 'Tokens.Transfer'"
            f"    and args->>'to' = '0x6d6f646c6f6d6e69706f6f6c0000000000000000000000000000000000000000'"
            f"  union all"
            f"  select"
            f"    block_id,"
            f"    args->>'currencyId' as asset_id,"
            f"    -(args->>'amount')::numeric as amount"
            f"  from event"
            f"  where"
            f"    name = 'Tokens.Transfer'"
            f"    and args->>'from' = '0x6d6f646c6f6d6e69706f6f6c0000000000000000000000000000000000000000'"
            f"  union all"
            f"  select"
            f"    block_id,"
            f"    args->>'currencyId' as asset_id,"
            f"    (args->>'amount')::numeric as amount"
            f"  from event"
            f"  where"
            f"    name = 'Tokens.Deposited'"
            f"    and args->>'who' = '0x6d6f646c6f6d6e69706f6f6c0000000000000000000000000000000000000000'"
            f"  union all"
            f"  select"
            f"    block_id,"
            f"    args->>'currencyId' as asset_id,"
            f"    -(args->>'amount')::numeric as amount"
            f"  from event"
            f"  where"
            f"    name = 'Tokens.Withdrawn'"
            f"    and args->>'who' = '0x6d6f646c6f6d6e69706f6f6c0000000000000000000000000000000000000000'"
            f"),"
            f"balance_changes as ("
            f"  select * from hdx_changes"
            f"  union all"
            f"  select * from tokens_changes"
            f"),"
            f"balance_history as ("
            f"  select"
            f"    height,"
            f"    timestamp,"
            f"    block_id,"
            f"    asset_id,"
            f"    symbol,"
            f"    sum(amount) over (partition by asset_id order by block_id) / 10 ^ decimals as balance"
            f"  from balance_changes"
            f"  inner join block on block_id = block.id"
            f"  inner join token_metadata on asset_id = token_metadata.id::text"
            f")"
            f"select timestamp, symbol, balance as liquidity "
            f"from balance_history "
            f"order by timestamp asc "
            f"limit {chunk_size} offset {chunk_size} * {position}"
        )
    }
    new_data = []
    response = requests.post(
        url='https://sqlpad.play.hydration.cloud/api/batches',
        headers=headers,
        data=json.dumps(request)
    )

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data = response.json()
        # pprint(data)
        batchID = data['statements'][0]['batchId']
        # print(batchID)
        try:
            statement = requests.get(
                url=f'https://sqlpad.play.hydration.cloud/api/batches/{batchID}/statements',
                headers=headers
            ).json()
            # print("Statement:")
            # pprint(statement)
            statementID = statement[0]['id']
            # pprint(statementID)

            print(f'waiting for query page {position + 1}...')
            if response.status_code == 200:
                new_data = {'title': 'Not found'}
                while 'title' in new_data and new_data['title'] == 'Not found':
                    new_data = requests.get(
                        url=f'https://sqlpad.play.hydration.cloud/api/statements/{statementID}/results',
                        headers=headers
                    ).json()
                    time.sleep(1)
                print("finished.")
                # print(new_data)
                # tag a record number to each entry so we can go back and see if anything is missing
                new_data = [[position * chunk_size + i] + new_data[i] for i in range(len(new_data))]
                # insert at the correct position
                all_data = all_data[:position * chunk_size] + new_data + all_data[position * chunk_size:]
                # print('remaining errors:', check_errors(all_data))
                # pprint(data)
                    # with open('./data/omnipool_history', 'a') as file:
                    #     file.write((', ' if position > 0 else '') + ', '.join([json.dumps(line) for line in data]))

        except Exception as e:
            print(f"There was a problem with your request: {str(e)}")
    else:
        pprint(response)
    return all_data


def load_history_file(filename: str):
    with open(f'./data/{filename}', 'r') as file:
        file_data = json.loads('[' + file.read() + ']')
        index = file_data[-1][0]
    return file_data


def save_history_file(all_data: list, n: int):
    filename = f'./data/omnipool_history_{str(n).zfill(2)}'
    with open(filename, 'w') as file:
        file.write(', '.join([json.dumps(line) for line in all_data[chunk_size * chunks_per_file * (n - 1): chunk_size * chunks_per_file * n]]))
    print(f'Saved {filename}')


In [18]:
all_data = []
n = 0
file_ls = os.listdir('./data')
for filename in file_ls:
    if filename.startswith('omnipool_history'):
        print(f'loading {filename}')
        all_data += load_history_file(filename)

def fix_errors(all_data):
    # error checking and correction
    # this works in the specific case where a piece of chunk_size length failed to download, which is typical
    # other types of errors would require different handling
    errors = 1
    while errors:
        errors = check_errors(all_data)
        print(f'Detected error at: {errors[0]}')
        all_data = add_data(position=error, all_data=all_data)
    else:
        print('Data looks error-free.')

In [26]:
while True:
    fix_errors(all_data)
    file_number = round(len(all_data) / chunk_size / chunks_per_file) + 1
    new_data = []
    start_at = round(len(all_data) / chunk_size)
    for n in range(start_at, start_at + chunks_per_file):
        all_data = add_data(position=n, all_data=all_data)
    print(f'saving omnipool_history_{str(file_number).zfill(2)}')
    save_history_file(all_data, file_number)
    if all_data[-1][0] == len(all_data) - 1 and len(all_data) % chunk_size != 0:
        # probably means we're finished. There might be a better way to detect this but I think it'll do
        return all_data

Data looks error-free.
waiting for query page 263...
finished.
waiting for query page 264...


KeyboardInterrupt: 

In [32]:
all_data[-1][0], len(all_data)

(2621924, 2621925)

In [27]:
len(all_data) / chunk_size

262.1925

In [ ]:
save_history_file(all_data, 2)

In [ ]:
balances = {}
for line in all_data:
    symbol = line[1]
    if symbol not in balances:
        balances[symbol] = {}
    balances[symbol][line[0]] = line[-1]
for symbol in balances:
    plt.figure(figsize=(20, 5))
    plt.plot(balances[symbol].keys(), balances[symbol].values())
    plt.title(symbol)

In [ ]:
len(all_data)

In [ ]:
all_data[:200]

In [ ]:
[symbol for symbol in balances]

In [ ]:
1 + 1

In [ ]:
pprint(all_data[9990:10010])